In [1]:
#install pytorch, transformers, requests, beautifulsoup4, pandas, numpy


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [23]:
tokens = tokenizer.encode("GREAT!! it's a lot uncomfortable", return_tensors='pt')

In [24]:
result = model(tokens)

In [25]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.4821, -1.6231,  0.0590,  1.6307,  1.9625]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [26]:
result.logits

tensor([[-2.4821, -1.6231,  0.0590,  1.6307,  1.9625]],
       grad_fn=<AddmmBackward0>)

In [ ]:
int(torch.argmax(result.logits))+1

Collect reviews

In [35]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2?osq=mejico')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

Load reviews into DF & score

In [43]:
import pandas as pd
import numpy as np

In [48]:
df = pd.DataFrame(np.array(reviews),columns=['review'])

In [49]:
df

,review
0,"Great atmosphere, attentive service, solid mar..."
1,Don't come here expecting legit Mexican food b...
2,Out of all the restaurants that I tried in Syd...
3,The food is fresh and tasty. The scallop cevi...
4,We came here on a Thursday night @ 5pm and by ...
5,Have been here twice and have absolutely loved...
6,I was pleasantly surprised at what a great job...
7,If you're looking for a quiet little romantic ...
8,The service at this place was top notch - the ...
9,Really nice (upmarket) Mexican restaurant. Goo...


In [50]:
def sentiment_score(review) :
    tokens = tokenizer.encode(review,return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [51]:
sentiment_score(df['review'].iloc[0])

3

In [52]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [54]:
df

,review,sentiment
0,"Great atmosphere, attentive service, solid mar...",3
1,Don't come here expecting legit Mexican food b...,3
2,Out of all the restaurants that I tried in Syd...,5
3,The food is fresh and tasty. The scallop cevi...,4
4,We came here on a Thursday night @ 5pm and by ...,4
5,Have been here twice and have absolutely loved...,5
6,I was pleasantly surprised at what a great job...,5
7,If you're looking for a quiet little romantic ...,2
8,The service at this place was top notch - the ...,5
9,Really nice (upmarket) Mexican restaurant. Goo...,4


In [55]:
df.to_csv('final-sentiment.csv')